<a href="https://colab.research.google.com/github/davehusen/site/blob/main/%5BTEMPLATE%5D_Waste_Characterization_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use this notebook:

1. Make a copy of this notebook
2. Adjust the runtime variables as necessary
2. Run the setup steps to setup the colab instance environment
3. Execute a single run or hyperparameter sweep

# Runtime variables

In [1]:
repository_url = "git@github.com:orange-industrial/pipeline.git"  # Replace with your repository URL
branch = "dave/oi-177-add-ci-reorg-pipeline-code" # Replace with your branch name
pipeline = "waste-characterization" # Replace with your pipeline

# Setup the environment

In [ ]:
!pip install wandb -qqq
!pip install ultralytics --quiet
!pip install roboflow

In [3]:
import os
import subprocess
import shutil
from google.colab import drive

In [ ]:
drive = drive.mount("/content/drive")

In [ ]:
# Copy deploy key to instance for authentication to github repo

ssh_key_path = "/content/drive/MyDrive/Orange Industrial/Engineering/Colab Deploy Keys/" # Set the SSH key path
ssh_key_name = 'id_ed25519_pipeline_deploy' # Github read-only deploy key for repo at repository_url

! mkdir -p /root/.ssh
shutil.copy2(ssh_key_path+ssh_key_name, "/root/.ssh/id_ed25519")
! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
! chmod go-rwx /root/.ssh/
! ssh -T git@github.com

In [ ]:
! git clone git@github.com:orange-industrial/pipeline /content/pipeline

In [ ]:
! cd pipeline/ ; git checkout "$branch"

In [8]:
os.chdir("/content/pipeline/pipelines/" + pipeline)

# Run the training code

To execute a single run using `run_config.json` params:
```
!python main.py --rf_api_key $API_KEY
```

To execute a sweep using `sweep_config.json` params:
```
!python main.py --rf_api_key $API_KEY \
--sweep_flag True \
--sweep_json config/sweep_config.json
```


## Execute a single run

In [ ]:
!git pull
!python main.py --rf_api_key DooHVuDzvvLAevjx9YIt --wandb_api_key ed9e154fbfc5ef1458bc77b312183e4cbd097648 --model_name yolov8x

### Test inference

In [39]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [40]:
save_dir = "/content/drive/MyDrive/Orange Industrial/Engineering/Models/waste-characterization-3/devout-wildflower-99"
model_path = "/content/drive/MyDrive/Orange Industrial/Engineering/Models/waste-characterization-3/devout-wildflower-99/deploy_bundle/waste-characterization-devout-wildflower-99-backbone_yolov8x-epochs_100-batch_32-workers_8-lr_0.000891-20230807-213103.pt"

In [ ]:
test_data_dir = "/content/drive/MyDrive/Orange Industrial/Engineering/Datasets/waste-characterization-3/test/images"
list_of_test_files = os.listdir(test_data_dir)
print(list_of_test_files)

In [44]:
model = YOLO(model_path)

In [ ]:
img = os.path.join(test_data_dir,list_of_test_files[0])
res = model(img)
res_plotted = res[0].plot()
cv2_imshow(res_plotted)

## Execute a sweep

In [ ]:
!git pull
!python main.py --rf_api_key DooHVuDzvvLAevjx9YIt --wandb_api_key ed9e154fbfc5ef1458bc77b312183e4cbd097648 \
--sweep_flag True \
--sweep_json config/sweep_config.json